In [1]:
# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import math
import torch

# show HTML in notebook
import IPython

from keybert import KeyBERT

from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
import re
from typing import List, Tuple, Union

import spacy

import inspect
import os# disabiguated_reference_liit = np.load('disambiguated_reference_list.npy')

import ollama
from tqdm import tqdm
import time

import spacy
spacy.load('en_core_web_sm')

import pandas as pd
import sys

module_path = 'core'
sys.path.append(module_path)

from SemanticSimilarity import SemanticSimilarity
from KG import KG
from KeywordExtractor import KeywordExtractor
from Search import Search
from Evaluation import Evaluation


from py2neo import Graph, Node, Relationship
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)     # Display all rows

C:\Users\zawad\miniconda3\envs\phd\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
questions_from_hotpot_1 = [
    "When was Radio City, India's first private FM radio station, started?",
    "What are the frequencies at which Radio City broadcasts in Mumbai and other cities?",
    "Who is the CEO of Radio City?",
    "When was the Albanian National Football Team founded, and when did it play its first international match?",
    "What is Echosmith best known for, and when was their debut album 'Talking Dreams' released?",
    "What is significant about Salem College and Wesleyan College in the context of women's colleges in the Southern United States?",
    "What distinction does the First Arthur County Courthouse and Jail hold in the United States?",
    "Who were some of the contributors to Arthur's Magazine, and what happened to it in May 1846?",
    "How many teams participated in the 2014–15 Ukrainian Hockey Championship, and why was this number so low?",
    "What was the sequence of events leading to the Freeway Complex Fire in 2008, and how many residences were ultimately destroyed?"
]

In [3]:
answers = [
    "Radio City was started on 3 July 2001.",
    "Radio City broadcasts on 91.1 megahertz (earlier 91.0 in most cities).",
    "Abraham Thomas is the CEO of Radio City.",
    "The Albanian National Team was founded on June 6, 1930, and played its first international match in 1946.",
    "Echosmith is best known for their hit song 'Cool Kids,' and their debut album 'Talking Dreams' was released on October 8, 2013.",
    "Salem College is the oldest female educational institution in the South, and Wesleyan College is the first that was established specifically as a college for women.",
    "The First Arthur County Courthouse and Jail is perhaps the smallest courthouse in the United States and now serves as a museum.",
    "Contributors to Arthur's Magazine included Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, and Thomas G. Spear. In May 1846, it was merged into Godey's Lady's Book.",
    "Only four teams participated in the league this season due to instability in Ukraine and economic issues faced by most of the clubs.",
    "The Freeway Complex Fire started as two separate fires on November 15, 2008: the 'Freeway Fire' shortly after 9 am and the 'Landfill Fire' approximately two hours later. These fires merged a day later, ultimately destroying 314 residences in Anaheim Hills and Yorba Linda."
]

In [4]:
e = Evaluation()

In [5]:
# disambiguated_reference_list = np.load('disambiguated_reference_list_hotpot_4.npy')
# embedded_reference = np.load('embedded_reference_kg_demo_hotpot_4.npy')
# xs = SemanticSimilarity()
# xs.embedded_reference = embedded_reference
# xs.disambiguated_reference_list = disambiguated_reference_list

# kwe = KeywordExtractor()

In [6]:
kg_triplets = np.load('hotpot_4_triplets.npy')
subs = []
for i in kg_triplets:
    subs.append(i[0])
subs = list(set(subs))
dem_s = SemanticSimilarity()

dem_s.generate_reference_without_disambiguation(subs)

C:\Users\zawad\miniconda3\envs\phd\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [7]:
search_obj = Search(dem_s, kwe, graph)


In [8]:
search_obj.retrieve_summary("Sports: Ukrainian Hockey Championship 2014-2015")

[]

In [9]:
# ollama.generate(model="mistral", prompt="What distinction does the First Arthur County Courthouse and Jail hold in the United States?")

{'model': 'mistral',
 'created_at': '2024-06-24T23:29:47.0984216Z',
 'response': " The First Arthur County Courthouse and Jail, located in Keene, Nebraska, is significant historically as it is the oldest surviving courthouse in Nebraska. It was built in 1871, making it one of the earliest structures of its kind in the state. This courthouse has served as a symbol of the area's growth and development since its construction, playing a crucial role in the administration of justice during the settlement of the region. As such, it holds historical importance for the local community and the state of Nebraska.",
 'done': True,
 'done_reason': 'stop',
 'context': [3,
  29473,
  2592,
  21445,
  2003,
  1040,
  4973,
  12953,
  5836,
  5841,
  1130,
  2052,
  1072,
  1243,
  1382,
  3168,
  1065,
  1040,
  3737,
  4311,
  29572,
  29473,
  4,
  29473,
  1183,
  4973,
  12953,
  5836,
  5841,
  1130,
  2052,
  1072,
  1243,
  1382,
  29493,
  6419,
  1065,
  5470,
  2628,
  29493,
  22133,
  294

In [9]:
x = search_obj.get_label_relevant_triplets("What distinction does the First Arthur County Courthouse and Jail hold in the United States?")

In [10]:
x

['Jail was perhaps smallest court house',
 'Jail was court house',
 'Jail was perhaps smallest court house in United States',
 'Jail serves now as museum',
 'Jail serves as museum',
 'nThe First Arthur County Courthouse was smallest court house in United States',
 'nThe First Arthur County Courthouse component  The text discusses various subjects:\n1. The song "Es" by an unspecified band, released by Warner Bros. Records, was one of the top-selling digital songs in 2014 with over 1.3 million downloads and achieved success in both the United States and Australia.\n2. Women\'s colleges in the Southern United States are undergraduate institutions that were historically all-female or nearly so, many of which originated as girls\' seminaries or academies. Examples include Salem College, the oldest female educational institution in the South, and Wesleyan College, the first college for women. Some schools offer coeducational courses at the graduate level, such as Mary Baldwin University and 

In [44]:
label = "First courthouse in the country"

In [21]:
q = "Who is the CEO of Radio City?"
label_relevant_triplets = search_obj.get_label_relevant_triplets(q)
answer = ollama.generate(model="mistral", prompt=f"Given the source text {label_relevant_triplets}, analyse it and give an answer to the question {q}")["response"]
print(q)
print(answer)
print(e.check_answer_semantically_similar(answer, "Abraham Thomas"))

[]
Who is the CEO of Radio City?
 The provided source text does not contain any information about the CEO of Radio City. Therefore, it's impossible to determine who the CEO of Radio City based on this information alone. You may want to refer to reliable sources such as the company's official website or other credible online resources for accurate information.
False


In [11]:
def answer_questions(questions, answers):
    for i,q in enumerate(questions):
        label_relevant_triplets = search_obj.get_label_relevant_triplets(q)
        print(label_relevant_triplets)
        answer = ollama.generate(model="mistral", prompt=f"Given the source text {label_relevant_triplets}, analyse it and give an answer to the question {q}")["response"]
        print(q)
        print(answer)
        print(e.check_answer_semantically_similar(answer, answers[i]))

In [12]:
answer_questions(questions_from_hotpot_1, answers)

['Radio City recently forayed in May 2008', 'Radio City component  Radio City is India\'s first private FM radio station, launched on July 3, 2001. It broadcasts in Hindi, English, and regional languages across various cities including Mumbai, Bengaluru, Lucknow, New Delhi, Hyderabad, Chennai, and Visakhapatnam. The station began operations in multiple cities at different times, with the first broadcast starting in Bengaluru in 2001, followed by Mumbai in 2004. Abraham Thomas is the CEO of the company.\n\nFootball in Albania predates the Albanian Football Federation (FSHF), as evidenced by the team\'s registration at the Balkan Cup tournament during 1929-1931. Despite facing pressure from other teams due to competition, football activities started first and were robust enough for duels. The Albanian National Team was founded on June 6, 1930, but had to wait until 1946 to play its first international match against Yugoslavia, which they won. Albania joined FIFA in 1932 during the 12th-1

In [19]:
search_obj.get_answers("What distinction does the First Arthur County Courthouse and Jail hold in the United States?")

The verbs in the question are ['hold']
these are the keywords
['arthur county courthouse', 'jail', 'the united states']
the graph entity is
nThe First Arthur County Courthouse
the graph entity is
Jail
the graph entity is
India
[]
Texts



' The First Arthur County Courthouse and Jail holds a significant distinction in the United States as it is believed to be the first courthouse in the country to have been built of concrete. This historical building, located in Arthur, Nebraska, was completed in 1887, marking an innovative step in construction materials at that time.'